# Laboratorium 7

Celem siódmego laboratorium jest zapoznanie się oraz zaimplementowanie algorytmu głębokiego uczenia aktywnego - Actor-Critic. Zaimplementowany algorytm będzie testowany z wykorzystaniem środowiska z OpenAI - *CartPole*.


Dołączenie standardowych bibliotek

In [1]:
from collections import deque
import gym
import numpy as np
import random

Dołączenie bibliotek do obsługi sieci neuronowych

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.math import log
from tensorflow.math import reduce_sum
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

## Zadanie 1 - Actor-Critic

<p style='text-align: justify;'>
Celem ćwiczenie jest zaimplementowanie algorytmu Actor-Critic. W tym celu należy utworzyć dwie głębokie sieci neuronowe:
    1. *actor* - sieć, która będzie uczyła się optymalnej strategii (podobna do tej z laboratorium 6),
    2. *critic* - sieć, która będzie uczyła się funkcji oceny stanu (podobnie jak się DQN).
Wagi sieci *actor* aktualizowane są zgodnie ze wzorem:
\begin{equation*}
    \theta \leftarrow \theta + \alpha \delta_t \nabla_\theta log \pi_{\theta}(a_t, s_t | \theta).
\end{equation*}
Wagi sieci *critic* aktualizowane są zgodnie ze wzorem:
\begin{equation*}
    w \leftarrow w + \beta \delta_t \nabla_w\upsilon(s_{t + 1}, w),
\end{equation*}
gdzie:
\begin{equation*}
    \delta_t \leftarrow r_t + \gamma \upsilon(s_{t + 1}, w) - \upsilon(s_t, w).
\end{equation*}
</p>

In [3]:
class Agent:
    def __init__(self, state_size, action_size, actor, critic, policy_model):
        self.state_size = state_size
        self.action_size = action_size
        self.action_space = [i for i in range(action_size)]
        self.gamma = 0.99    # discount rate
        self.learning_rate = 0.001
        self.policy_model = policy_model
        self.actor = actor
        self.critic = critic #critic network should have only one output


    def choose_action(self, state):
        """
        Compute the action to take in the current state, basing on policy returned by the network.

        Note: To pick action according to the probability generated by the network
        """

        #
        # INSERT CODE HERE to get action in a given state
        #
        state = state[np.newaxis, :]
        probabilities = self.policy_model.predict(state, verbose = 0)[0]
        chosen_action = np.random.choice(self.action_space, p=probabilities)        
        
        return chosen_action

  

    def learn(self, state, action, reward, next_state, done):
        """
        Function learn networks using information about state, action, reward and next state. 
        First the values for state and next_state should be estimated based on output of critic network.
        Critic network should be trained based on target value:
        target = r + \gamma next_state_value if not done]
        target = r if done.
        Actor network shpuld be trained based on delta value:
        delta = target - state_value
        """
        #
        # INSERT CODE HERE to train network
        #

        state = state[np.newaxis, :]
        next_state = next_state[np.newaxis, :]
        critic_value_next_state = self.critic.predict(next_state)
        critic_value = self.critic.predict(state)

        target = reward + self.gamma * critic_value_next_state * (1 - int(done))
        delta = target - critic_value

        actions = np.zeros([1, self.action_size])
        actions[np.arange(1), action] = 1

        self.actor.fit([state, delta], actions, verbose=0)
        self.critic.fit(state, target, verbose=0)  
        


Czas przygotować model sieci, która będzie się uczyła działania w środowisku [*CartPool*](https://gym.openai.com/envs/CartPole-v0/):

In [4]:
env = gym.make("CartPole-v1").env
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
alpha_learning_rate = 0.0001
beta_learning_rate = 0.0005

input = Input(shape=(state_size,))
delta = Input(shape=[1])
x1 = Dense(64, activation='relu')(input)
probs = Dense(action_size, activation='softmax')(x1)
values = Dense(1, activation='linear')(x1)

policy_model = Model(inputs=[input], outputs=[probs])

def custom_loss(y_true, y_pred):
    log_lik = y_true * log(y_pred)
    return reduce_sum(-log_lik * delta)

#
# INSERT CODE HERE to build actor network, in the last layer use softmax activation function
#

actor_model = Model(inputs=[input, delta], outputs=[probs])
actor_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr=alpha_learning_rate), loss=custom_loss)
       
#
# INSERT CODE HERE to build critic network, in the last layer use linear activation function, network should have single output
#

critic_model = Model(inputs=[input], outputs=[values])
critic_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr=beta_learning_rate), loss='mean_squared_error')

c:\Users\Bartycja\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Czas nauczyć agenta gry w środowisku *CartPool*:

In [5]:
agent = Agent(state_size, action_size, actor_model, critic_model, policy_model)


for i in range(100):
    score_history = []

    for i in range(100):
        done = False
        score = 0
        state = env.reset()[0]
        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, info, _ = env.step(action)
            agent.learn(state, action, reward, next_state, done)
            state = next_state
            score += reward
        score_history.append(score)

    print("mean reward:%.3f" % (np.mean(score_history)))

    if np.mean(score_history) > 300:
        print("You Win!")
        break

c:\Users\Bartycja\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


mean reward:13.560
mean reward:15.810
mean reward:21.780
mean reward:36.820
mean reward:49.920
mean reward:87.760
mean reward:122.420
mean reward:130.330
mean reward:142.780
mean reward:126.610
mean reward:99.810
mean reward:147.430
mean reward:139.830
mean reward:123.310
mean reward:172.640
mean reward:290.270
mean reward:168.730
mean reward:204.850
mean reward:2027.140
You Win!
